In [25]:
import json
from urllib.request import Request, urlopen
import ssl
from pathlib import Path

In [15]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [35]:
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:102.0) Gecko/20100101 Firefox/102.0",
}

In [49]:
album_id = "c820920f-b92c-4353-9627-ce1c5699fd9a"
batch_size = 100

In [ ]:
registry = []
while True:
    req = Request(
        f"https://mysport.photo/public/photo-service/api/v1/photos?albumId={album_id}&offset={len(registry)}&count={batch_size}",
        headers=headers,
    )
    res = urlopen(req, context=ctx)
    data = json.loads(res.read())
    total = data["total"]
    registry += data["values"]
    print(f"registered {len(registry)} of {total}")
    if len(registry) >= total:
        break

In [52]:
with open("registry.json", "w") as f:
    f.write(json.dumps(registry))

In [53]:
with open("registry.json", "r") as f:
    registry = json.loads(f.read())

In [54]:
base_dir = Path("photos")
base_dir.mkdir(exist_ok=True)

In [ ]:
for i, item in enumerate(registry):
    path = base_dir / (item["photoId"] + ".jpg")
    if not path.exists():
        with open(path, "wb") as f:
            data = urlopen(Request(item["resources"]["watermark"], headers=headers), context=ctx)
            f.write(data.read())
    print(f"downloaded {i + 1} of {len(registry)}")